# Vader Sentiment Analysis
This notebook takes you through doing sentiment analysis of tweets and exporting the data. You can then visualize the results as a bar graph using the "vaderSentimentResults" notebook.

For more information about how Vader works behind the scenes see here: https://github.com/cjhutto/vaderSentiment

###  Before we begin
Before we start, you will need to have set up a [Carbonate account](https://kb.iu.edu/d/aolp) in order to access [Research Desktop (ReD)](https://kb.iu.edu/d/apum). You will also need to have access to ReD through the [thinlinc client](https://kb.iu.edu/d/aput). If you have not done any of this, or have only done some of this, but not all, you should go to our [textPrep-Py.ipynb](https://github.com/cyberdh/Text-Analysis/blob/drafts/textPrep-Py.ipynb) before you proceed further. The textPrep-Py notebook provides information and resources on how to get a Carbonate account, how to set up ReD, and how to get started using the Jupyter Notebook on ReD.   

### Run CyberDH environment
The code in the cell below points to a Python environment specifically for use with the Python Jupyter Notebooks created by Cyberinfrastructure for Digital Humanities. It allows for the use of the different packages in our notebooks and their subsequent data sets.

##### Packages
- **sys:** Provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter. It is always available.
- **os:** Provides a portable way of using operating system dependent functionality.

#### NOTE: This cell is only for use with Research Desktop. You will get an error if you try to run this cell on your personal device!!

In [1]:
import sys
import os
sys.path.insert(0,"/N/u/cyberdh/Carbonate/dhPyEnviron/lib/python3.6/site-packages")
os.environ["NLTK_DATA"] = "/N/u/cyberdh/Carbonate/dhPyEnviron/nltk_data"

### Include necessary packages for notebook 

Python's extensibility comes in large part from packages. Packages are groups of functions, data, and algorithms that allow users to easily carry out processes without recreating the wheel. Some packages are included in the basic installation of Python, others created by Python users are available for download.

In your terminal, packages can be installed by simply typing `pip install nameofpackage --user`. However, since you are using ReD and our Python environment, you will not need to install any of the packages below to use this notebook. Anytime you need to make use of a package, however, you need to import it so that Python knows to look in these packages for any functions or commands you use. Below is a brief description of the packages we are using in this notebook:     

- **nltk:** Platform for building Python programs to work with human language data. Here we bring in the VADER sentiment analysis tool which is now a part of the nltk package.

- **pickle:** Implements binary protocols for serializing and de-serializing a Python object structure. "Pickling" is the process whereby a Python object hierarchy is converted into a byte stream, and "unpickling" is the inverse operation, whereby a byte stream (from a binary file or bytes-like object) is converted back into an object hierarchy.

- **pandas:** An open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

- **os:** This module provides a portable way of using operating system dependent functionality.

- **glob:** Finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order.

- **zipfile:** Allows for handling of zipfiles.

Notice we import some of the packages differently. In some cases we just import the entire package when we say `import XYZ`. For some packages which are small, or, from which we are going to use a lot of the functionality it provides, this is fine. 

Sometimes when we import the package directly we say `import XYZ as X`. All this does is allow us to type `X` instead of `XYZ` when we use certain functions from the package. So we can now say `X.function()` instead of `XYZ.function()`. This saves time typing and eliminates errors from having to type out longer package names. I could just as easily type `import XYZ as potato` and whenever I use a function from the `XYZ` package I would need to type `potato.function()`. What we import the package as is up to you, but some commonly used packages have abbreviations that are standard amongst Python users such as `import pandas as pd` or `import matplotlib.pyplot as plt`. You do not need to us `pd` or `plt`, however, these are widely used and using something else could confuse other users and is generally considered bad practice. 

Other times we import only specific elements or functions from a package. This is common with packages that are very large and provide a lot of functionality, but from which we are only using a couple functions or a specific subset of the package that contains the functionality we need. This is seen when we say `from XYZ import ABC`. This is saying I only want the `ABC` function from the `XYZ` package. Sometimes we need to point to the specific location where a function is located within the package. We do this by adding periods in between the directory names, so it would look like `from XYZ.123.A1B2 import LMN`. This says we want the `LMN` function which is located in the `XYZ` package and then the `123` and `A1B2` directory in that package. 

You can also import more than one function from a package by separating the functions with commas like this `from XYZ import ABC, LMN, QRS`. This imports the `ABC`, `LMN` and `QRS` functions from the `XYZ` package.

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import pickle
import pandas as pd
import os
import glob
import zipfile

### Variables
Here we create some variables for use later in our code. We do this to minimize the number and complexity of the changes you will need to make later.

First we need to decide if we want to read in one file or a whole directory of files. If you want to read in a whole directory then set `source` equal to `"*"` as this is what is called a regular expression that means 'all'. So we are reading in 'all' the files in a directory. If you wish to read in a single file then set `source` equal to the name of the desired file in quotes, but leaving the '.json' or '.csv' off at the end. So a single file should look like this: `source = "myFileName"`.

Next we assign the file type our data comes in to a variable. At the moment the only options are '.csv' or '.json' as these are the most popular twitter data formats. We assign the format to the `fileType` variable. It should look like this: `fileType = ".json"`.

The `textColIndex` variable is where we put the header name of the dataframe column that will contain the content we are interested in from our tweets. Generally the content of the tweets are labeled as "text" since this is the label given to the tweet content when it is pulled directly from the Twitter API. For this reason our default value assigned to the `textColIndex` is `"text"`. If for some reason the tweet content has a different label or header, and you need to change this, remember to keep the quotes around the new label.

The `remove` variable is assigned a boolean of either **True** or **False**. If it is **True** it means that you want to "remove" the terms in the `remWords` list below from the vader lexicon. If you set it to **False**, you do not have any words to remove.

The `add` variable is assigned a boolean of either **True** or **False**. If it is **True** it means you want add the key/value pairs in the dictionary `newWords` to the vader lexicon. The `newWords` dcitionary (just below the `add` boolean variable) contains **"word": vader polarity score** for words you would like to add to the vader lexicon. The scores in the example dictionary were made up (did not follow vader protocol), however, if you wanted to add terms you would need to follow a similar protocol to vader and find ten people to score the word between -4 (most negative) and 4 (most positive) including 0 as a possible score, and then get the average score (which is the number after each word in the `newWords` dictionary) and also determine the standard deviation as the creators of vader did not include words that had a standard deviation of over 2.5.   

The variable `encoding` is where you determine what type of encoding to use (ascii, ISO-8850-1, utf-8, etc...). We have it set to utf-8 at the moment as we have found it is less likely to have any problems.

The remaining variables should not need to be changed. The variable `scores` is an empty list that will have data added to it further down in the code. The variables `total`, `numberOfTweets`, and `totalSquared` are variables used for counting which is why 0 is assigned to them since we generally start with nothing when counting.

In [3]:
source = "coronaVirus01-21Jan2020"
fileType = ".json"
textColIndex = "text"
remove = True
remWords = ["novel", "ha", "l", "gt"]
add = True
newWords = {"virus": -1.7, "outbreak": -0.6, "epidemic": -2.3, "pandemic": -3.1, "quarantine": -2.6}
encoding = "utf-8"
scores = []
total = 0
numberOfTweets = 0
totalSquared = 0

### File paths
Here we assign file paths we will need throughout this notebook to variables. This way we only need to make changes here and they will be implemeneted throughout the code. The `homePath` variable uses the `environ` function from the `os` package. This function points to your home directory no matter your operating system (Linux, osX, Windows).

Then we join the `homePath` variable to folders that point to where our data is stored and we assign this file path to the variable `dataHome`. The folder names are in quotes and separated by a comma. 

Finally, we again use the `homePath` variable and join it with a file path that points to a folder where we can save our data output. We assign this file path to a variable called `dataClean`.

You can change any of these to better match where your data can be found (`dataHome`) and where you want any output such as '.csv' files or images to be saved (`dataClean`).

In [4]:
homePath = os.environ['HOME']
vaderPath = "/N/u/cyberdh/Carbonate/dhPyEnviron/nltk_data/sentiment"
dataHome = os.path.join(homePath, "Text-Analysis-master","data","twitter")
newVaderWords = os.path.join(dataHome, "sentiment", "newVaderWords.txt")
dataClean = os.path.join(homePath,"Text-Analysis-master","VADERSentimentAnalysis", "cleanedData")

### Unzip files

Here we are unzipping files. Since twitter data can be rather large it is often necessary to compress it into a '.zip' file in order to upload it to places such as GitHub. For this reason, we have setup some code to go in and automatically extract all the items in a compressed '.zip' file so you don't have to and so you don't get errors later. If the data is not in a '.zip' file there is no need to worry, it will not give an error if there are no files ending in '.zip' in your directory.

The only changes you may wish to make are in the first two lines after `if fileType == ".csv":` (if you set `fileType` equal to `".csv"` above) or the first two lines after `else:` (if you set `fileType` equal to `".json"` up above. These are the lines that point to the file paths where your '.zip' files are stored. If you have '.zip' files stored in another folder you will want to change the path. Note that the first line points to the directory and the second line points to the files.  

In [5]:
if fileType == ".csv":
    direct = os.path.join(dataHome, "CSV")
    allZipFiles = glob.glob(os.path.join(dataHome, "CSV","*.zip"))
    for item in allZipFiles:
            fileName = os.path.splitext(direct)[0]
            zipRef = zipfile.ZipFile(item, "r")
            zipRef.extractall(fileName)
            zipRef.close()
            os.remove(item)
else:
    direct = os.path.join(dataHome, "JSON")
    allZipFiles = glob.glob(os.path.join(dataHome, "JSON","*.zip"))
    for item in allZipFiles:
            fileName = os.path.splitext(direct)[0]
            zipRef = zipfile.ZipFile(item, "r")
            zipRef.extractall(fileName)
            zipRef.close()
            os.remove(item)
if add == True:
    vDirect = vaderPath
    vaderZipFile = glob.glob(os.path.join(vaderPath, "*.zip"))
    for item in vaderZipFile:
        vaderFile = os.path.splitext(vDirect)[0]
        vaderRef = zipfile.ZipFile(item, "r")
        vaderRef.extractall(vaderFile)
        vaderRef.close()
        os.remove(item)

### Reading in .csv and .json files

If you chose `".csv"` as your `fileType` up above, then the first `if` statement in the code below reads in ".csv" files and saves the contents to a dataframe using the Pandas package. It will read in either an entire directory or a single ".csv" file depending on what you chose for `source` above. 

Once we have read in the ".csv" file using the Pandas `read_csv` function, we need to concatenate the ".csv" files if there are multiple. Because of this it is important that your ".csv" files have an identical column count and each column has identical header names or you will get errors. If you have a single ".csv" file then you should be fine for this step. We assign this process to the variable `cdf` so we can use it later.

Now we convert our `cdf` to a pandas dataframe. This allows for easier manipulation of the data in the next line.

Finally, we pull in the column containing the data we are interested in which we assigned to the variable `textColIndex` earlier and turn it into a list assigned to the variable `tweets`.

If you chose `".json"` for your fileType, then the second `if` statement will read in ".json" files and save the content to a dataframe using the Pandas package much like the ".csv" file process described above. The only difference is that we use the Pandas function `read_json` instead of `read_csv`. Everything else is exactly the same as what is described above in the ".csv" section. 

In [6]:
if fileType == ".csv":
    allFiles = glob.glob(os.path.join(dataHome, "CSV", source + fileType))     
    df = (pd.read_csv(f, engine = "python") for f in allFiles)
    cdf = pd.concat(df, ignore_index=True)
    cdf = pd.DataFrame(cdf)
    tweets = cdf[textColIndex].values.tolist()
if fileType == ".json":
    allFiles = glob.glob(os.path.join(dataHome, "JSON", source + fileType))     
    df = (pd.read_json(f, encoding = encoding, lines = True) for f in allFiles)
    cdf = pd.concat(df, ignore_index=True)
    cdf = pd.DataFrame(cdf)
    tweets = cdf[textColIndex].values.tolist()

Now we check to see if we have pulled our tweets from our dataset. We are just checking the first 10 tweets. If you wish to see more change the 10 in the parantheses to the number of tweets you wish to see. If you wish to see the last 10 tweets then change `rtDF.head(10)` to `rtDF.tail(10)`.

In [7]:
rtDF = pd.DataFrame(tweets)
rtDF.head(10)

,0
0,"Simply...si no és #coronavirus, NO hi ha #SARS..."
1,RT @XavierAbadMdG: Simply...si no és #coronavi...
2,Update on cluster of #Patients infected with #...
3,"#HK, Two suspected #MERS #Coronavirus cases re..."
4,Suspicion de Syndrome Respiratoire du Moyen-Or...
5,"#HK, Suspected #MERS #Coronavirus case reporte..."
6,"RT @ironorehopper: #HK, Suspected #MERS #Coron..."
7,"5th Jan Wuhan official report\n\n1. 59 cases, ..."
8,Latest reports are of a possible #SARS-#MERS h...
9,"#HK, Suspected #MERS #Coronavirus case reporte..."


#### Create labels for different sentiment values and start the count for each value at zero
Here we create the labels for our different sentiment values by creating a dictionary. The number in quotes is the "key" and the integer after the colon is the value. Our sentiment scores can be anywhere from -1 to 1 with -1 having a very negative sentiment and 1 having a very positive sentiment. We assign this dictionary to the variable `res`.

In [8]:
res = {"-1":0, "-.9":0, "-.8":0, "-.7":0, "-.6":0, "-.5":0, "-.4":0, "-.3":0, "-.2":0, "-.1":0, "0":0, ".1":0, ".2":0,".3":0, ".4":0, ".5":0, ".6":0, ".7":0, ".8":0, ".9":0, "1":0}

#### Shorten SentimentIntensityAnalyzer Function

We shorten the `SentimentIntensityAnalyzer()` by assigning it to the variable `vader`.

In [9]:
vader = SentimentIntensityAnalyzer()

#### Remove words
Here we have an "if...else" statement. If we assigned **True** to the variable `remove` above then we take each word in the `remWords` list and apply (`map`) the `.pop` function from vader to each word in the list. 

If we assigned **False** to `remove` then we do nothing.

In [10]:
if remove == True:
    map(vader.lexicon.pop, remWords)
else:
    None

#### Add words
Here we have another "if...else" statement. If we assigned **True** to `add` then we use the `.update` function from vader to add each `{"key": value}` pair from our `newWords` dictionary above.

If we assigned **False** to `add` then we do nothing.

In [11]:
if add == True:
    vader.lexicon.update(newWords)
else:
    None

#### Go through and apply the Vader sentiment analyzer to all tweets and count them

Here we apply VADER to our tweets and start adding the results to the `res` dictionary as well as the `total`, `numberOfTweets`, `totalSquared`, and `scores` variables. No changes should be needed here.

In [12]:
for index, row in rtDF.iterrows():
    vs = vader.polarity_scores(str(rtDF.iloc[:,0][index]))
    scores.append(vs['compound'])
    total += vs["compound"]
    numberOfTweets += 1
    totalSquared += vs["compound"]**2
    if vs["compound"]==0.0:
        res["0"] +=1
    elif 0 < vs["compound"] <= 0.1:
        res[".1"] +=1
    elif 0.1 <= vs["compound"] <= 0.2:
        res[".2"] +=1
    elif 0.2 < vs["compound"] <= 0.3:
        res[".3"] +=1
    elif 0.3 < vs["compound"] <= 0.4:
        res[".4"] +=1
    elif 0.4 < vs["compound"] <= 0.5:
        res[".5"] +=1
    elif 0.5 < vs["compound"] <= 0.6:
        res[".6"] +=1
    elif 0.6 < vs["compound"] <= 0.7:
        res[".7"] +=1
    elif 0.7 < vs["compound"] <= 0.8:
        res[".8"] +=1
    elif 0.8 < vs["compound"] <= 0.9:
        res[".9"] +=1
    elif 0.9 < vs["compound"] <= 1:
        res["1"] +=1
    elif 0 > vs["compound"] >= -0.1:
        res["-.1"] +=1
    elif -0.1 > vs["compound"] >= -0.2:
        res["-.2"] +=1
    elif -0.2 > vs["compound"] >= -0.3:
        res["-.3"] +=1
    elif -0.3 > vs["compound"] >= -0.4:
        res["-.4"] +=1
    elif -0.4 > vs["compound"] >= -0.5:
        res["-.5"] +=1
    elif -0.5 > vs["compound"] >= -0.6:
        res["-.6"] +=1
    elif -0.6 > vs["compound"] >= -0.7:
        res["-.7"] +=1
    elif -0.7 > vs["compound"] >= -0.8:
        res["-.8"] +=1
    elif -0.8 > vs["compound"] >= -0.9:
        res["-.9"] +=1
    elif -0.9 > vs["compound"] >= -1:
        res["-1"] +=1

print(res)

{'-1': 42, '-.9': 1996, '-.8': 218, '-.7': 230, '-.6': 582, '-.5': 1606, '-.4': 196, '-.3': 586, '-.2': 348, '-.1': 641, '0': 5107, '.1': 325, '.2': 364, '.3': 233, '.4': 1400, '.5': 342, '.6': 227, '.7': 185, '.8': 199, '.9': 50, '1': 10}


### Export data

Now we use the pickle package to export the data needed by the "vaderSentimentResults" notebook to create the bar graph visualization. We save the data to the folder in the file path assigned to the `dataClean` variable. To change the names of the output files change the contents in the quotes such as `"vaderScores"`, `"total"`, `"scores"`, `"numberOfText"`, and `"squared"`. DO NOT change `"wb"` as this states that we are writing a bytes object to file which is what we want to do.

In [13]:
with open(os.path.join(dataClean, "vaderScores"), "wb") as vaderScore:
    pickle.dump(res, vaderScore)
with open(os.path.join(dataClean, "total"), "wb") as vaderTotal:
    pickle.dump(total, vaderTotal)
with open(os.path.join(dataClean, "scores"), "wb") as s:
    pickle.dump(scores, s)
with open(os.path.join(dataClean, "numberOfText"), "wb") as nt:
    pickle.dump(numberOfTweets, nt)
with open(os.path.join(dataClean, "squared"), "wb") as squared:
    pickle.dump(totalSquared, squared)

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.